# IPython Notebook Validation for py.test - Documentation

One of the powerful uses of the IPython notebook is for documentation purposes, here we use a notebook to demonstrate the behaviour and usage of the IPython Notebook Validation plugin for py.test. The IPython notebook format `.ipynb` stores outputs as well as inputs. Validating the notebook means to rerun the notebook and make sure that it is generating the same output as has been stored.

Therefore, the **user MUST make the following the distinction**:

1. Running a notebook manually will likely change the output stored in the associated .ipynb file. These outputs will be used as references for the tests (i.e. the outputs from the last time you ran the notebook)
2. Validating with py.test plugin - these tests run your notebook code seperately without storing the information, the outputs generated will be compared against those in the .ipynb file

The purpose of the testing module is to ensure that the notebook is behaving as expected and that changes to underlying source code, haven't affected the results of an IPython notebook. For example, for documentation purposes - such as this.

### Command line usage

The py.test program doesn't usually collect notebooks for testing; by passing the `--ipynb` flag at the command line, the IPython Notebook Validation plugin will collect at test notebook cells.
```
$ py.test --ipynb my_notebook.ipynb
```

### REGEX Output sanitizing

Since all output is captured by the IPython notebook, some pesky messages and prompts (with time-stamped messages, for example) may fail tests always, which might be expected. The plugin allows the user to specify a sanitizing file at the command prompt using the following flag:
```
$ py.test --ipynb my_notebook.ipynb --sanitize-with my_sanitize_file
```

This sanitize file contains a number of REGEX replacements. It is recommended, when removing output for the tests, that you replace the removed output with some sort of marker, this helps with debugging. The following file is written to the folder of this notebook and can be used to santize its outputs:

In [1]:
%%writefile doc_sanitize.cfg
[regex1]
regex: \d{1,2}/\d{1,2}/\d{2,4}
replace: DATE-STAMP

[regex2]
regex: \d{2}:\d{2}:\d{2}
replace: TIME-STAMP

Writing doc_sanitize.cfg


The first replacement finds dates in the given format replaces them with the label 'DATE-STAMP', likewise for strings that look like time. These will prevent the tests from failing due to time differences.

### Validate this notebook

You can validate this notebook yourself, as shown below; the outputs that you see here are stored in the ipynb file. If your system produces different outputs, the testing process will fail. Just use the following commands:
```
$ cd /path/to/this/notebook
$ py.test --ipynb documentation.ipynb --sanitize-with doc_sanitize.cfg
```

### Examples of plugin behaviour

The following examples demonstrate how the plugin behaves during testing. Test this notebook yourself to see the validation in action!

These two imports produce no output as standard, if any **warnings** are printed out the cell will fail. Under normal operating conditions they will pass.

In [2]:
import numpy as np
import time

If python doesn't consistently print 7, then something has gone terribly wrong. **Deterministic cells** are expected to pass everytime

In [3]:
print 5+2

7


**Random outputs** will always fail.

In [4]:
print [np.random.rand() for i in range(4)]
print [np.random.rand() for i in range(4)]

[0.44541343532071564, 0.314205135593974, 0.9314621193930673, 0.42361058071747515]
[0.3332858303310847, 0.4163797894141835, 0.47729582938253023, 0.518940452299377]


**Inconsistent number of lines** of output will cause an error to be thrown.

In [5]:
for i in range(np.random.randint(1, 8)):
    print 1

1
1
1
1


Because the **time and date** will change with each run, we would expect this cell to fail everytime. Using the sanitize file `doc_sanitize.cfg` (created above) you can clean up these outputs.

In [6]:
print 'The time is: ' + time.strftime('%H:%M:%S')
print "Today's date is: " + time.strftime('%d/%m/%y')

The time is: 15:57:54
Today's date is: 20/02/15


### Avoid specific cell executions

In case we want to avoid the testing process in specific input cells, we can write the comment ** #PYTEST_VALIDATE_IGNORE_OUTPUT ** at the
beginning of the them:

In [7]:
#PYTEST_VALIDATE_IGNORE_OUTPUT
print 'This is not going to be tested'
print np.random.randint(1, 20000)

This is not going to be tested
947
